In [1]:
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from gensim.models import CoherenceModel

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\S\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\S\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
df = pd.read_csv('../ready_df.csv').dropna()

In [5]:
# Есть ли пропуски:
print(df.isnull().sum())
# Есть ли дубликаты
print(df.duplicated().any())
print(len(df))

Unnamed: 0                         0
ucid                               0
message                            0
ACTION_ITEM_RESULT_PRODUCT_NAME    0
dtype: int64
False
42620


In [6]:
df.head()

,Unnamed: 0,ucid,message,ACTION_ITEM_RESULT_PRODUCT_NAME
0,0,"5,01000641613474E+017",добрый день это клиентский менеджер виктория с...,Бизнес-карта
1,1,"5,01001121614156E+017",добрый день звать клиентский менеджер сбербанк...,Зарплатные проекты
2,2,"5,01001811614178E+017",добрый день звать сбербанк блок сбер бизнес об...,Зарплатные проекты
3,3,"5,01002531614256E+017",добрый день клиентский менеджер удобно разгова...,Бизнес-карта
4,4,"5,01005041613457E+017",добрый день это сбербанк блок сбер бизнес клие...,Зарплатные проекты


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df.message, test_size=0.3, random_state=42)

In [42]:
vectorizer = CountVectorizer(
    max_features=3000,
    analyzer='word',
    min_df=10,
    ngram_range=(2, 3),
    stop_words=stopwords.words('russian')
)
vectorizer.fit(X_train)

CountVectorizer(max_features=3000, min_df=10, ngram_range=(2, 3),
                stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с',
                            'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
                            'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы',
                            'по', 'только', 'ее', 'мне', ...])

In [43]:
train = vectorizer.transform(X_train).toarray()
test = vectorizer.transform(X_test).toarray()

In [44]:
lda = LatentDirichletAllocation(
    n_components=16,
    learning_method='online',
    random_state=42,
    n_jobs=-1
)
lda.fit(train)

LatentDirichletAllocation(learning_method='online', n_components=16, n_jobs=-1,
                          random_state=42)

In [46]:
def get_coherence_mean(model, texts, n_top_words=20):
    # Кол-во тем:
    topics = model.components_

    # Предикт:
    texts = [[word for word in doc.split()] for doc in texts]

    # Словарь
    dictionary = corpora.Dictionary(texts)

    # Корпус на основе матрицы подсчета слов
    corpus = [dictionary.doc2bow(text) for text in texts]
    feature_names = [dictionary[i] for i in range(len(dictionary))]

    # Получение ТОП слов из каждой темы:
    top_words = []
    for topic in topics:
        top_words.append([feature_names[i] for i in topic.argsort()[:-n_top_words:-1]])

    coherence_model = CoherenceModel(corpus=corpus,
                                     topics=top_words,
                                     texts=texts,
                                     dictionary=dictionary,
                                     coherence='c_v')
    coherence = coherence_model.get_coherence()
    return coherence

In [47]:
get_coherence_mean(lda, X_test)

0.5474537721452609

# Подобр кол-ва тематик оптимального

In [49]:
def predict_topic(data, model=lda):
    topic_proba = model.transform(data)[0]
    if np.all(topic_proba == topic_proba[0]):
        return -17
    return np.argmax(topic_proba)

In [50]:
df['topics'] = [predict_topic(string) for string in vectorizer.transform(df['message'])]

In [51]:
vocab = vectorizer.get_feature_names_out()

for i, comp in enumerate(lda.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
личный кабинет сбер бизнес сбербанк бизнес бизнес онлайн добрый свидание расчётный счёт отделение банк мочь обращаться оставаться линия сбербанк бизнес онлайн 

Topic 1: 
услуга банк ваш время рада помочь уделить время свой контакт продукт услуга расчётный счёт персональный менеджер хотеть уточнить сказать пожалуйста 

Topic 2: 
блок сбер сбер бизнес блок сбер бизнес сбербанк блок добрый день сбербанк блок сбер бизнес звонить сбер бизнес звонить звонить организация клиентский менеджер 

Topic 3: 
минута ваш минута ваш время ваш время хотеться обсудить который хотеться предложение который который хотеться обсудить несколько минута время предложение предложение который хотеться 

Topic 4: 
бизнес карта готовность карта тысяча рубль рубль сутки тысяча рубль сутки личный кабинет рабочий день течение календарный денежный средство комиссия снятие 

Topic 5: 
сказать пожалуйста юридический лицо обслуживание юридический обслуживание юридический лицо добрый день торговый эквайринг удо

In [57]:
from gensim import corpora


# Предикт:
texts = [[word for word in doc.split()] for doc in X_train]

# Словарь
dictionary = corpora.Dictionary(texts)

# Корпус на основе матрицы подсчета слов
corpus = [dictionary.doc2bow(text) for text in texts]

In [58]:
# Словарь с ключевыми словами для каждой темы
topics_keywords = {
    'Приветствие': ['здравствуйте', 'привет', 'добрый день', 'рады видеть', 'приветствуем', 'приветствие', 'добро пожаловать'],
    'Перенос активности': ['перенос', 'перемещение', 'переход', 'изменение места', 'активность', 'перемещение дел', 'перевод'],
    'Окончание разговора': ['до свидания', 'спасибо за разговор', 'удачи', 'завершение', 'прощание'],
    'Болтовня': ['разговор', 'беседа', 'болтовня', 'бесполезный разговор', 'пустая болтовня'],
    'Следующие шаги': ['дальнейшие действия', 'следующий этап', 'план действий', 'следующие шаги', 'последующие мероприятия'],
    'Безопасность': ['безопасность', 'защита', 'безопасный', 'защищенный', 'безопасность данных'],
    'Тарифы / Цены': ['тарифы', 'цены', 'стоимость', 'тарифные планы', 'расценки'],
    'Акции': ['акции', 'специальные предложения', 'скидки', 'акционные предложения', 'распродажа'],
    'Удобство работы': ['удобство', 'комфорт', 'удобное использование', 'удобство в эксплуатации', 'легкость использования'],
    'Простота настройки': ['простота', 'легкость', 'простая настройка', 'простота установки', 'быстрая настройка'],
    'Скорость работы': ['скорость', 'быстрота', 'оперативность', 'скорость выполнения', 'быстрая работа'],
    'Преимущества': ['преимущество', 'плюсы', 'позитивные стороны', 'достоинства', 'преимущества использования'],
    'Вовлечение': ['вовлечение', 'заинтересованность', 'подключение к диалогу', 'стимулирование обсуждения'],
    'Призыв к действию': ['призыв к действию', 'призыв к действию клиента', 'мотивация к действию', 'призыв к выполнению'],
    'Инструкция по подключению/приобретению': ['инструкция', 'руководство', 'подробное описание', 'пошаговая инструкция', 'указания по установке/приобретению'],
    'Вопрос клиента': ['вопрос', 'запрос', 'просьба', 'требование', 'вопрос клиента']
}

In [ ]:
from gensim.models import LdaModel

num_topics = len(topics_keywords)  # количество тем равно количеству ключевых слов
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

AttributeError: 'LdaModel' object has no attribute 'add_topic'

In [60]:
# Вывод тем и соответствующих им ключевых слов
for idx, topic in lda_model.print_topics(-1):
    print(f'Topic: {idx} \nWords: {topic}')

Topic: 0 
Words: 0.061*"организация" + 0.048*"добрый" + 0.033*"день" + 0.031*"просить" + 0.029*"звонить" + 0.027*"прощение" + 0.027*"сбербанк" + 0.027*"номер" + 0.025*"мочь" + 0.025*"бизнес"
Topic: 1 
Words: 0.059*"карта" + 0.031*"бизнес" + 0.024*"наличный" + 0.022*"ваш" + 0.019*"покупка" + 0.018*"предложение" + 0.017*"этот" + 0.017*"банк" + 0.015*"счёт" + 0.015*"год"
Topic: 2 
Words: 0.067*"карта" + 0.064*"бизнес" + 0.031*"лимит" + 0.023*"банк" + 0.021*"сумма" + 0.017*"мочь" + 0.014*"балл" + 0.014*"тысяча" + 0.014*"кэшбэк" + 0.013*"сбер"
Topic: 3 
Words: 0.036*"вопрос" + 0.026*"услуга" + 0.026*"время" + 0.025*"звонить" + 0.023*"ваш" + 0.023*"добрый" + 0.019*"день" + 0.018*"сбербанк" + 0.017*"быть" + 0.017*"менеджер"
Topic: 4 
Words: 0.042*"личный" + 0.042*"кабинет" + 0.041*"код" + 0.027*"смс" + 0.025*"бизнес" + 0.023*"подключение" + 0.022*"мочь" + 0.021*"зайти" + 0.019*"телефон" + 0.018*"инструкция"
Topic: 5 
Words: 0.019*"это" + 0.018*"нужно" + 0.016*"нажимать" + 0.012*"счёт" + 0.012

In [64]:
# Функция для получения топика из текста
def get_topic(text):
    # tokens = preprocess_text(text)
    bow_vector = dictionary.doc2bow(text.split())
    topics_distribution = lda_model.get_document_topics(bow_vector)
    # Возвращаем номер топика с наибольшей вероятностью для данного документа
    return max(topics_distribution, key=lambda item: item[1])[0]

# Добавление столбца с номером топика для каждой строки
df['g_topics'] = df['message'].apply(get_topic)